In [33]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
import statsmodels.formula.api as smf

In [3]:
data = pd.read_excel("/content/drive/MyDrive/data/chemitech.xlsx")

In [4]:
data

,A,B,C
0,58,58,48
1,64,69,57
2,55,71,59
3,66,64,47
4,67,68,49


In [7]:
data_r = pd.melt(data.reset_index(), id_vars=["index"], value_vars=["A", "B", "C"])
data_r.columns = ["index", "treatments", "value"]

In [8]:
data_r

,index,treatments,value
0,0,A,58
1,1,A,64
2,2,A,55
3,3,A,66
4,4,A,67
5,0,B,58
6,1,B,69
7,2,B,71
8,3,B,64
9,4,B,68


In [10]:
model = ols("value ~ C(treatments)", data=data_r).fit()
anova_table = sm.stats.anova_lm(model, type = 1)
anova_table

,df,sum_sq,mean_sq,F,PR(>F)
C(treatments),2.0,520.0,260.000000,9.176471,0.003818
Residual,12.0,340.0,28.333333,NaN,NaN


In [12]:
just_dummies = pd.get_dummies(data_r["treatments"])
just_dummies

,A,B,C
0,True,False,False
1,True,False,False
2,True,False,False
3,True,False,False
4,True,False,False
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False


In [36]:
step_1 = pd.concat([data_r, just_dummies], axis=1)
step_1.drop(["treatments", "C"], inplace=True, axis=1)
step_1

,index,value,A,B
0,0,58,True,False
1,1,64,True,False
2,2,55,True,False
3,3,66,True,False
4,4,67,True,False
5,0,58,False,True
6,1,69,False,True
7,2,71,False,True
8,3,64,False,True
9,4,68,False,True


In [37]:
result = smf.ols(step_1["value"], sm.add_constant(step_1[["A", "B"]])).fit()
print(result.summary())

PatsyError: model is missing required outcome variables

In [38]:
result = smf.ols("value ~ A + B", data=step_1).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  value   R-squared:                       0.605
Model:                            OLS   Adj. R-squared:                  0.539
Method:                 Least Squares   F-statistic:                     9.176
Date:                Sat, 05 Apr 2025   Prob (F-statistic):            0.00382
Time:                        16:57:12   Log-Likelihood:                -44.691
No. Observations:                  15   AIC:                             95.38
Df Residuals:                      12   BIC:                             97.51
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     52.0000      2.380     21.844      0.0

/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  res = hypotest_fun_out(*samples, **kwds)
